<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep_with_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.layers import *
from keras.optimizers import *
from keras.models import *
from keras import backend as K
import keras
import random
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

import gc


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [3]:

trn=pd.read_csv('/content/gdrive/My Drive/fraud/train.csv')
tst=pd.read_csv('/content/gdrive/My Drive/fraud/test.csv')
trn=trn.drop(['isFraud'],1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (210,222) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cats=list(trn.select_dtypes(include=object))

In [5]:
dum=list(trn.filter(regex='dum'))
cat=list(trn.select_dtypes(include=object))
isna=list(trn.filter(regex='isna'))
tot=dum+cat+isna+['date','month','week']
num=[ i for i in list(trn) if i not in tot]
for col in num:
  trn[col+'_mean']=trn.groupby(['id'])[col].transform('mean')
  trn[col+'_std']=trn.groupby(['id'])[col].transform('std')
  
  tst[col+'_mean']=tst.groupby(['id'])[col].transform('mean')
  tst[col+'_std']=tst.groupby(['id'])[col].transform('std')
trn=trn.drop(['id'],1)
tst=tst.drop(['id'],1)


In [6]:


class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
cols=list(trn.select_dtypes(include=object))
for col in tqdm(cols):
  le=LabelEncoderExt()
  le.fit(trn[col].astype(str))
  trn[col]=le.transform(trn[col].astype(str))
  tst[col] = tst[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  tst[col]=le.transform(tst[col].astype(str))

100%|██████████| 21/21 [01:15<00:00,  3.60s/it]


In [7]:

cats+=list(trn.filter(regex='dum'))
no_dum=[i for i in list(trn) if i not in cats]

In [8]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
trn=reduce_mem_usage(trn)
tst=reduce_mem_usage(tst)

Memory usage of dataframe is 4491.95 MB
Memory usage after optimization is: 1168.04 MB
Decreased by 74.0%
Memory usage of dataframe is 3854.15 MB
Memory usage after optimization is: 1022.97 MB
Decreased by 73.5%


In [9]:

np.random.seed(42) # NumPy
random.seed(42) # Python

In [10]:

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [11]:
from keras.layers import concatenate
from keras.optimizers import Adam
def create_model():
    K.clear_session()
    num_inp = Input(shape=(num_shape,))
    cat_inp = Input(shape=(cat_shape,))
    inps = concatenate([num_inp, cat_inp])
    x = Dense(512, activation=custom_gelu)(inps)
    x = Dense(256, activation=custom_gelu)(x)
    x = Dense(512, activation = custom_gelu)(x)
    x = Dropout(.2)(x)
    cat_out = Dense(cat_shape, activation = "linear")(x)
    num_out = Dense(num_shape, activation = "linear")(x)
    model = Model(inputs=[num_inp,cat_inp], outputs=[num_out, cat_out])
    model.compile(
        optimizer=Adam(.05, clipnorm = 1, clipvalue = 1),
        loss=["mse", "mse"]
    )
    return model

In [12]:
X=pd.concat([trn,tst],0)
del([trn,tst])
gc.collect()

33

In [13]:
cats.remove('id')

Make the final dataset

In [14]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
numerical=pd.DataFrame(ss.fit_transform(X[no_dum]))
numerical.columns=no_dum
categorical=pd.DataFrame(ss.fit_transform(X[cats]))
categorical.columns=cats
X=pd.concat([categorical,numerical],1)
X=reduce_mem_usage(X)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.6/dist-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


Memory usage of dataframe is 7460.84 MB
Memory usage after optimization is: 2105.36 MB
Decreased by 71.8%


In [15]:

from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)

In [16]:
from keras.utils import Sequence
class DAESequence(Sequence):
    def __init__(self,df,no_dum,frac=0.15,dumm=range(911),batch_size=2048):
        self.batch_size=batch_size
        self.frac=0.15
        self.dumm=dumm
        self.df=df
        self.cat_data=df[dumm].values
        self.num_data=df[no_dum].values
        self.no_dumm=no_dum
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df
        self.idx=[]
        
        
        
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    
    
    
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        
        
        inps=[]
        outs=[]
        output_x=self.data.iloc[idx:last]
        
        
        
        
        data=output_x[self.no_dumm].values
        noise_x=data.copy()
        for i in range(len(self.no_dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
            
        inps.append(noise_x)
        outs.append(data)
        
        data=output_x[self.dumm].values
        noise_x=data.copy()
        for i in range(len(self.dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
        
        
        
        inps.append(noise_x)
        outs.append(data)
        
        
        
        
        
        return inps,outs

Fill the std columns

In [17]:
from tqdm import tqdm
a=X.isna().sum()
a=a[a>0]
cls=list(X)
for col in tqdm(list(a.index)):
  if col in cls:
    X[col]=X[col].fillna(X[col].mean())

a=X.isna().sum()
a=a[a>0]
cls=list(X)
for col in tqdm(list(a.index)):
  X=X.drop([col],1)

100%|██████████| 32/32 [01:04<00:00,  2.02s/it]


In [18]:
tot.remove('id')
tot.remove('date')
tot.append('day')

In [ ]:

num=[ i for i in list(X) if i not in tot]
batch_size=2048
auto_ckpt = ModelCheckpoint("ae.model", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', period=1)
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.0001)
num_shape=len(num)
cat_shape=len(tot)
model_mse = create_model()
from math import *
gc.collect()
epochs = 1000
batch_size=2048
train_gen=DAESequence(X,num,batch_size=batch_size,dumm=tot)
hist = model_mse.fit_generator(train_gen, steps_per_epoch=len(X)//batch_size, epochs=epochs,
                           verbose=1, workers=-1,
                           use_multiprocessing=True,
                              callbacks=[auto_ckpt, warm_up_lr])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1000
535/535 [==============================] - ETA: 0s - loss: 1.5811 - dense_4_loss: 0.7812 - dense_3_loss: 0.7998
Epoch 00001: loss improved from inf to 1.58106, saving model to ae.model
535/535 [==============================] - 42s 79ms/step - loss: 1.5811 - dense_4_loss: 0.7812 - dense_3_loss: 0.7998
Epoch 2/1000
535/535 [==============================] - ETA: 0s - loss: 1.0221 - dense_4_loss: 0.5475 - dense_3_loss: 0.4746
Epoch 00002: loss improved from 1.58106 to 1.02206, saving model to ae.model
535/535 [==============================] - 41s 77ms/step - loss: 1.0221 - dense_4_loss: 0.5475 - dense_3_loss: 0.4746
Epoch 3/1000
535/535 [==============================] - ETA: 0s - loss: 0.7935 - dense_4_loss: 0.4509 - dense_3_loss: 0.3426
Epoch 00003: loss improved from 1.02206 to 0.79352, saving model to ae.model
535/535 [==============================] - 41s 76ms/step - loss: 0.7935 - dense_4_loss

In [ ]:
gc.collect()

In [ ]:
mod=Model(inputs=model_mse.inputs,outputs=model_mse.layers[4].output)
mod.summary()

In [ ]:
pre=mod.predict((X[num],X[tot]))
pre.shape

In [ ]:
np.save('/content/gdrive/My Drive/fraud/with_id.npy',pre)